## В данном ноутбуке происходит labelling акций, join сентимента и цен

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

### Сентимент

In [2]:
s = pd.read_csv('/is/texts/mag/mag_text_lower_tic_all_flag_all_dt_e_reg_stop_stem_len.csv', sep='|')
s.head()

,date,ticker,total_msg,text_reg_stop,text_reg_stop_stem,len_lem_text
0,20210311.0,sber,2548,схема крупной программы борьбы бедностью рф по...,схем крупн программ борьб бедност рф поддержк ...,4041
1,20140108.0,aflt,916,следственный комитет россии скр начал проверку...,следствен комитет росс скр нача проверк эвитер...,2642
2,20140108.0,sber,2548,владелец ратма таран продал основной цементный...,владелец ратм тара прода основн цементн акт гр...,2387
3,20140108.0,gazp,2391,прошлом году увеличил экспорт частности европу...,прошл год увелич экспорт частност европ сократ...,1630
4,20140109.0,zill,90,председатель совета директоров независимой неф...,председател совет директор независим нефтян ко...,961


In [3]:
print(s.shape)
s = s[(s.text_reg_stop_stem != '')]
print(s.shape)
s = s[(s.text_reg_stop_stem.isna() != True)]
print(s.shape)
s = s.drop_duplicates()
print(s.shape)

(16280, 6)
(16280, 6)
(16280, 6)
(16280, 6)


In [4]:
s['msg_init'] = 1
s.head()

,date,ticker,total_msg,text_reg_stop,text_reg_stop_stem,len_lem_text,msg_init
0,20210311.0,sber,2548,схема крупной программы борьбы бедностью рф по...,схем крупн программ борьб бедност рф поддержк ...,4041,1
1,20140108.0,aflt,916,следственный комитет россии скр начал проверку...,следствен комитет росс скр нача проверк эвитер...,2642,1
2,20140108.0,sber,2548,владелец ратма таран продал основной цементный...,владелец ратм тара прода основн цементн акт гр...,2387,1
3,20140108.0,gazp,2391,прошлом году увеличил экспорт частности европу...,прошл год увелич экспорт частност европ сократ...,1630,1
4,20140109.0,zill,90,председатель совета директоров независимой неф...,председател совет директор независим нефтян ко...,961,1


### Цена

In [5]:
p = pd.read_csv('/is/prices/mag_price_change.csv', sep='|')
p.shape

(77298, 26)

In [6]:
p.head()

,ticker,per,date,time,open,high,low,close,vol,init,...,change_close_7,change_close_8,change_close_9,change_close_10,change_close_15,change_close_20,change_close_25,change_close_30,change_close_40,change_close_50
0,afks,NaN,20150208.0,NaN,16.475,17.8600,16.2700,17.0500,50174450.0,0.0,...,11.344697,8.649621,6.992424,4.598485,6.045455,-9.640152,-4.583333,1.181818,-0.600379,-4.047348
1,afks,NaN,20150329.0,NaN,17.120,17.3850,16.8500,17.3500,11615400.0,0.0,...,-2.472427,-5.155139,-4.844196,-3.745266,-3.192147,-7.463125,-6.194771,-5.304299,-2.218125,2.997143
2,afks,NaN,20150524.0,NaN,16.990,17.1150,16.8450,16.9300,4198500.0,0.0,...,0.501710,0.861733,3.049194,4.639627,8.948887,9.872863,16.925720,10.504536,19.436955,23.215582
3,afks,NaN,20151122.0,NaN,17.360,17.7250,17.1850,17.5300,8739450.0,0.0,...,1.366253,3.060999,2.913499,0.066747,-2.594427,-1.174646,-0.364635,3.279086,0.560050,0.703665
4,afks,NaN,20151213.0,NaN,16.985,17.1875,16.7875,17.0725,6941850.0,0.0,...,3.618088,3.397399,3.075960,2.283046,6.033962,5.397489,3.272946,3.881582,2.506257,2.870410


In [7]:
s.date = s.date.astype(float)
p.date = p.date.astype(float)
s.ticker = s.ticker.str.lower()
p.ticker = p.ticker.str.lower()

In [8]:
print(s.shape, p.shape)

(16280, 7) (77298, 26)


In [9]:
p.rename(columns = {'init': 'price_init'}, inplace=True)
p.head()

,ticker,per,date,time,open,high,low,close,vol,price_init,...,change_close_7,change_close_8,change_close_9,change_close_10,change_close_15,change_close_20,change_close_25,change_close_30,change_close_40,change_close_50
0,afks,NaN,20150208.0,NaN,16.475,17.8600,16.2700,17.0500,50174450.0,0.0,...,11.344697,8.649621,6.992424,4.598485,6.045455,-9.640152,-4.583333,1.181818,-0.600379,-4.047348
1,afks,NaN,20150329.0,NaN,17.120,17.3850,16.8500,17.3500,11615400.0,0.0,...,-2.472427,-5.155139,-4.844196,-3.745266,-3.192147,-7.463125,-6.194771,-5.304299,-2.218125,2.997143
2,afks,NaN,20150524.0,NaN,16.990,17.1150,16.8450,16.9300,4198500.0,0.0,...,0.501710,0.861733,3.049194,4.639627,8.948887,9.872863,16.925720,10.504536,19.436955,23.215582
3,afks,NaN,20151122.0,NaN,17.360,17.7250,17.1850,17.5300,8739450.0,0.0,...,1.366253,3.060999,2.913499,0.066747,-2.594427,-1.174646,-0.364635,3.279086,0.560050,0.703665
4,afks,NaN,20151213.0,NaN,16.985,17.1875,16.7875,17.0725,6941850.0,0.0,...,3.618088,3.397399,3.075960,2.283046,6.033962,5.397489,3.272946,3.881582,2.506257,2.870410


In [10]:
ps = p.merge(s, how='inner', on=['date', 'ticker'])
ps.to_csv('/is/prices/mag_price_sentiment_join.csv', sep='|', index = False)
print(ps.shape)
ps.head()

(15281, 31)


,ticker,per,date,time,open,high,low,close,vol,price_init,...,change_close_20,change_close_25,change_close_30,change_close_40,change_close_50,total_msg,text_reg_stop,text_reg_stop_stem,len_lem_text,msg_init
0,afks,NaN,20150208.0,NaN,16.475,17.860,16.2700,17.050,50174450.0,0.0,...,-9.640152,-4.583333,1.181818,-0.600379,-4.047348,182,коррупционный скандал разгорелся подконтрольно...,коррупцион сканда разгорел подконтрольн минстр...,3820,1
1,afks,NaN,20150329.0,NaN,17.120,17.385,16.8500,17.350,11615400.0,0.0,...,-7.463125,-6.194771,-5.304299,-2.218125,2.997143,182,федеральная таможенная служба фтс изменила пра...,федеральн таможен служб фтс измен прав беспошл...,3637,1
2,afks,NaN,20150524.0,NaN,16.990,17.115,16.8450,16.930,4198500.0,0.0,...,9.872863,16.925720,10.504536,19.436955,23.215582,182,следственный комитет рф вчера предъявил обвине...,следствен комитет рф вчер предъяв обвинен окон...,1550,1
3,afks,NaN,20151122.0,NaN,17.360,17.725,17.1850,17.530,8739450.0,0.0,...,-1.174646,-0.364635,3.279086,0.560050,0.703665,182,стало известно ъ совет евразийской экономическ...,стал известн ь совет евразийск экономическ ком...,3445,1
4,afks,NaN,20160110.0,NaN,17.800,18.050,17.4825,17.835,7370300.0,0.0,...,-2.245219,-1.948543,-0.822827,2.681120,-1.711137,182,продала детского мира российскокитайскому инве...,прода детск мир российскокитайск инвестицион ф...,1811,1


In [11]:
print(ps.shape)
ps = ps[ps.ticker.isin(p.ticker.unique())]
print(ps.shape)

(15281, 31)
(15281, 31)


In [19]:
for i in ps.columns: print(f"'{i}', ", sep='', end='')

'ticker', 'per', 'date', 'time', 'open', 'high', 'low', 'close', 'vol', 'price_init', 'change_close_1', 'change_close_2', 'change_close_3', 'change_close_4', 'change_close_5', 'change_close_6', 'change_close_7', 'change_close_8', 'change_close_9', 'change_close_10', 'change_close_15', 'change_close_20', 'change_close_25', 'change_close_30', 'change_close_40', 'change_close_50', 'total_msg', 'text_reg_stop', 'text_reg_stop_stem', 'len_lem_text', 'msg_init', 'label_1_0.05', 'label_1_0.1', 'label_1_0.2', 'label_1_0.3', 'label_1_0.5', 'label_1_0.8', 'label_1_1.0', 'label_1_2.0', 'label_1_3.0', 'label_1_4.0', 'label_1_5.0', 'label_1_6.0', 'label_1_7.0', 'label_1_8.0', 'label_1_9.0', 'label_1_10.0', 'label_2_0.05', 'label_2_0.1', 'label_2_0.2', 'label_2_0.3', 'label_2_0.5', 'label_2_0.8', 'label_2_1.0', 'label_2_2.0', 'label_2_3.0', 'label_2_4.0', 'label_2_5.0', 'label_2_6.0', 'label_2_7.0', 'label_2_8.0', 'label_2_9.0', 'label_2_10.0', 'label_3_0.05', 'label_3_0.1', 'label_3_0.2', 'label_3_

In [13]:
ps.to_csv('/is/prices/mag_price_sentiment_join.csv', sep='|', index = False)

In [14]:
def labelling(df, price, field, n=0):
#исправить функцию чтобы она по массиву брала
    
    '''
    Функция определяет сентимент поля price
    и указывает в новом поле field:
    1 для положительного фона,
    0 для негативного фона,
    -1 для нейтрального фона.
    Фон определяется на основе уровня изменения цены - n.
    
    '''
    df[field] = None
    
    df[field][df[price] >= (n)] = 1
    df[field][df[price] <= (-n)] = -1
    df[field][(df[price] > (-n)) & (df[price] < (n))] = 0


In [36]:
np.linspace(1, 10, num=10)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [15]:
yields = [0.05, 0.1, 0.2, 0.3, 0.5, 0.8, 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]
lags = [1,2,3,4,5,6,7,8,9,10, 15, 20, 25, 30, 40, 50]
for lag in tqdm(lags, position=0, leave=True): 
    for income in tqdm(yields, position=0, leave=True):
        labelling(df=ps, price=f'change_close_{lag}', field=f'label_{lag}_{income}', n=income)

ps.to_csv('/is/prices/mag_join.csv', sep='|', index = False)
ps.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ticker,per,date,time,open,high,low,close,vol,price_init,...,label_50_1.0,label_50_2.0,label_50_3.0,label_50_4.0,label_50_5.0,label_50_6.0,label_50_7.0,label_50_8.0,label_50_9.0,label_50_10.0
0,afks,NaN,20150208.0,NaN,16.475,17.860,16.2700,17.050,50174450.0,0.0,...,-1,-1,-1,-1,0,0,0,0,0,0
1,afks,NaN,20150329.0,NaN,17.120,17.385,16.8500,17.350,11615400.0,0.0,...,1,1,0,0,0,0,0,0,0,0
2,afks,NaN,20150524.0,NaN,16.990,17.115,16.8450,16.930,4198500.0,0.0,...,1,1,1,1,1,1,1,1,1,1
3,afks,NaN,20151122.0,NaN,17.360,17.725,17.1850,17.530,8739450.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,afks,NaN,20160110.0,NaN,17.800,18.050,17.4825,17.835,7370300.0,0.0,...,-1,0,0,0,0,0,0,0,0,0


In [20]:
ps.drop(columns=['per', 'time', 'open', 'high', 'low', 
       'text_reg_stop', 'msg_init'], inplace=True)

In [21]:
ps.to_csv('/is/prices/mag_join_lab.csv', sep='|', index = False)

In [22]:
ps.head()

,ticker,date,close,vol,price_init,change_close_1,change_close_2,change_close_3,change_close_4,change_close_5,...,label_50_1.0,label_50_2.0,label_50_3.0,label_50_4.0,label_50_5.0,label_50_6.0,label_50_7.0,label_50_8.0,label_50_9.0,label_50_10.0
0,afks,20150208.0,17.050,50174450.0,0.0,4.015152,4.770833,6.306818,9.248106,14.081439,...,-1,-1,-1,-1,0,0,0,0,0,0
1,afks,20150329.0,17.350,11615400.0,0.0,1.725799,2.739851,2.593681,2.307986,1.873630,...,1,1,0,0,0,0,0,0,0,0
2,afks,20150524.0,16.930,4198500.0,0.0,-0.263485,0.061510,0.208325,0.000977,-0.264462,...,1,1,1,1,1,1,1,1,1,1
3,afks,20151122.0,17.530,8739450.0,0.0,0.899222,0.894145,1.265936,1.001126,0.653419,...,0,0,0,0,0,0,0,0,0,0
4,afks,20160110.0,17.835,7370300.0,0.0,-1.479535,-1.535208,0.798060,0.887084,-2.264758,...,-1,0,0,0,0,0,0,0,0,0
